In [17]:
#import cvxpy as cvx
#import cvxopt
import networkx as nx
import numpy as np
import argparse
from typing import Tuple
# %matplotlib inline
import matplotlib.pyplot as plt
import random
from time import time
import os, sys
import subprocess
import re
import numpy as np
import pylab
import pickle
from qiskit import Aer, BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import ExactEigensolver, VQE
from qiskit.aqua.components.optimizers import SPSA, COBYLA, L_BFGS_B
from qiskit.aqua.components.variational_forms import RY, RYRZ
from qiskit.aqua.operators import Z2Symmetries
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.core import Hamiltonian, QubitMappingType, TransformationType
from qiskit.chemistry.drivers import PySCFDriver, UnitsType

import types

args = types.SimpleNamespace()
args.molecule='Be2' 
args.basis_set='ccpvdz' 
#args.basis_set='sto3g' 
args.algorithm='VQE' 
args.transformation_type='TransformationType.FULL' 
#args.qubitmapping_type='QubitMappingType.BRAVYI_KITAEV' 
#args.qubitmapping_type='bravyi_kitaev'
#args.qubitmapping_type='jordan_wigner'
args.qubitmapping_type='parity'
args.two_qubit_reduce = True
args.vqe_optimizer='SPSA' 
args.vqe_max_iter=2
args.vqe_var_form='RY' 
args.vqe_depth=1 
args.vqe_entangler='linear' 
args.num_shots=100 
args.max_parallel_threads=10 
args.vqe_sim = True
args.vqe_aer = True
args.datapath='/pylon5/cc5phsp/cbernaci/mol2qpu'
args.outpath='/home/ubuntu/mol2qpu/output/qubitOps'
args.random_seed=750 
args.vqe_opt_params = False




In [3]:
### READ IN MOLECULE
if args.molecule=='H2':
    molecule = 'H .0 .0 -{0}; H .0 .0 {0}'
elif args.molecule=='LiH':
    molecule = 'Li .0 .0 -{0}; H .0 .0 {0}'
elif args.molecule=='Be2':
    molecule = 'Be .0 .0 -{0}; Be .0 .0 {0}'

start = 0.5  # Start distance
steps = 0    # Number of steps to increase by
energies = np.zeros(steps+1)
hf_energies = np.zeros(steps+1)
distances = np.zeros(steps+1)
aqua_globals.random_seed = args.random_seed
d = start

driver = PySCFDriver(molecule.format(d/2), basis=args.basis_set)
qmolecule = driver.run()
ferOp = FermionicOperator(h1=qmolecule.one_body_integrals, h2=qmolecule.two_body_integrals)


num orbitals =  28
num alpha =  4
num beta =  4
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_bravyi_kitaev_mode', '_convert_to_block_spins', '_convert_to_interleaved_spins', '_h1', '_h1_transform', '_h2', '_h2_transform', '_jordan_wigner_mode', '_map_type', '_modes', '_one_body_mapping', '_parity_mode', '_ph_trans_shift', '_s_x_squared', '_s_y_squared', '_s_z_squared', '_two_body_mapping', 'fermion_mode_elimination', 'fermion_mode_freezing', 'h1', 'h2', 'mapping', 'modes', 'particle_hole_transformation', 'total_angular_momentum', 'total_magnetization', 'total_particle_number', 'transform']
['__abstractmethods__', '__add__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__e

In [14]:
num_particles = qmolecule.num_alpha + qmolecule.num_beta
qubitOp = ferOp.mapping(map_type=args.qubitmapping_type,threshold=0.00000001)
qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles)
#operator =  Hamiltonian(transformation=eval(args.transformation_type), 
#                        qubit_mapping=eval(args.qubitmapping_type),  
#                        two_qubit_reduction=args.two_qubit_reduce)

#print(dir(qmolecule))
print("num orbitals = ", qmolecule.num_orbitals)
#print("num alpha = ", qmolecule.num_alpha)
#print("num beta = ", qmolecule.num_beta)
print("num qubits = ", qubitOp.num_qubits)
#print(dir(ferOp))
print(dir(qubitOp))

num orbitals =  28
num qubits =  54
['__abstractmethods__', '__add__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__iadd__', '__init__', '__init_subclass__', '__isub__', '__le__', '__lt__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__sub__', '__subclasshook__', '__weakref__', '_abc_impl', '_add_or_sub', '_atol', '_basis', '_name', '_paulis', '_paulis_table', '_routine_compute_mean_and_var', '_scaling_weight', '_z2_symmetries', 'add', 'anticommute_with', 'atol', 'basis', 'chop', 'commute_with', 'construct_evaluation_circuit', 'copy', 'evaluate_with_result', 'evaluate_with_statevector', 'evaluation_instruction', 'evolve', 'evolve_instruction', 'from_dict', 'from_file', 'from_list', 'is_empty', 'multiply', 'name', 'num_qubits', 'paulis', 'print_details', 'reorder_paulis', 

In [19]:
print(args.outpath)

/home/ubuntu/mol2qpu/output/


In [20]:
bk_ccpvdz_2qr = qubitOp
filename = args.outpath + 'bk_ccpvdz_2qr.pkl'
filehandler = open(filename, 'wb')
pickle.dump(bk_ccpvdz_2qr, filehandler)

# where hist = some object
# and then you can reload with this
#bk_ccpvdz = pickle.load(open(filename,'rb'))

In [ ]:
#lines, result_op = operator.process_algorithm_result(result)

#qubitOp, aux_ops = operator.run(qmolecule)
#print(dir(operator))


#backend = Aer.get_backend('qasm_simulator')

#quantum_instance = QuantumInstance(circuit_caching=True, 
#                                   backend=backend,
#                                   backend_options={'max_parallel_threads': args.max_parallel_threads,                         #                                                    'max_parallel_experiments': 0, 
#                                                    'shots': args.num_shots})
## optimizer
#if args.vqe_optimizer=='SPSA':
#    optimizer = SPSA(max_trials=200)
#elif args.vqe_optimizer=='COBYLA':
#    optimizer = COBYLA()
#    optimizer.set_options(maxiter=args.vqe_max_iter)
#elif args.vqe_optimizer=='L_BFGS_B':
#    optimizer = L_BFGS_B(maxfun=args.vqe_max_iter)
#else:
#    optimizer = COBYLA()
#    optimizer.set_options(maxiter=args.vqe_max_iter)
#
### variational form
#if args.vqe_var_form=='RY':
#    var_form = RY(qubitOp.num_qubits, depth=args.vqe_depth, entanglement=args.vqe_entangler)   
#elif args.vqe_var_form=='RYRZ':
#    var_form = RYRZ(qubitOp.num_qubits, depth=args.vqe_depth, entanglement=args.vqe_entangler)
#
### VQE params
#if args.vqe_opt_params:
#    initial_point=np.load(args.vqe_opt_params_path+'._ret_opt_params.npy',allow_pickle=True, fix_imports=True)
#    algo = VQE(qubitOp, var_form, optimizer, initial_point=initial_point)
#else:
#    algo = VQE(qubitOp, var_form, optimizer)
#
#result = algo.run(quantum_instance)    
##result['algorithm_retvals']['eigvecs']
##results['eigvecs']
##type(results)
##eigvecs = results.get('eigvecs')
#
#
#circ_opt = algo.get_optimal_circuit()
#print(circ_opt)
#print(algo.print_settings())
#print('circuit_summary=', quantum_instance.circuit_summary)
#
#
#
#
#
##lines, resultA = operator.process_algorithm_result(results)
#
#
#
#


